# <span style="color:red">Rule inference with scBONITA</span>

## _Please refer to the README file for installation and setup instructions_

## <span style="color:blue">Step 1: Set up the scBONITA pipeline.</span>
scBONITA needs a training dataset in matrix-style forma; this is usually a tab or comma-delimited file with columns as cells and rows as features. The first column should be feature names and the first row should be cell IDs. The units of the expression data will typically be a variant of log2(TPM +1).



### The setup pipeline has the following parameters:

1. dataFile: Specify the name of the file containing processed scRNA-seq data
1. fullPipeline Should scBonita set up the entire pipeline, starting with generation of network topologies? Accepts values 0 or 1
1. network File name of the network for which rules are to be inferred
1. maxNodes Number of genes in the dataset
1. maxSamples Number of cells in the dataset
1. separator Delimiting character in dataFile. Must be one of , (comma), \s (space) or \t (tab)
1. getKEGGPathways Should scBonita automatically identify and download KEGG pathways with genes that are in your dataset? You can specify which pathways using the 
1. listOfKEGGPathways option, or leave it blank to download all matching KEGG pathways
1. listOfKEGGPathways Which KEGG pathways should scBonita download? Specify the five letter pathway IDs.
1. organism Three-letter organism code. Which organism is the dataset derived from?
1. cvThreshold: Minimum coefficient of variation to retain genes for scBONITA analysis



### The following command runs scBONITA setup for a 20000*10000 comma-separated data set "example.csv". It downloads the KEGG pathways hsa00010 and hsa00020 for rule inference.

`python3.6 pipeline.py --dataFile "example.csv" --fullPipeline 1 --maxNodes 20000 --maxSamples 10000 --separator "," --listOfKEGGPathways "00010" "00020" --getKEGGPathways 1 --organism hsa cvThreshold None`

## <span style="color:blue">Step 2A: Rule inference and calculation of node importance score for the networks specified in Step 1 (setup).</span>

#### Step 1 generates sbatch files that enter the specified slurm queue. In a typical use case, these jobs should execute automatically. We recommend that users periodically check the slurm queue and the log files.

## <span style="color:blue">Step 2B: Analysis of the output of scBONITA rule inference</span>

### Description of output files (for an example network 'hsa00010.graphml')

* **hsa00010_processed.graphml_out2.pickle**

    Representation of the model returned by the genetic algorithm component of scBONITA-RD
    

* **hsa00010_processed.graphml_rules_GA.txt**

    Plain-text representation of the model inferred by the genetic algorithm/global search component of scBONITA-rule determination


* **hsa00010_processed.graphml_rules_LS.txt**

    Plain-text representation of a randomly sampled set of rules from the equivalent rule set returned by the local search component of scBONITA-rule determination


* **hsa00010_processed.graphml_localErrors1.pickle**

    Values of the optimization function ("errors") associated with the ERS returned by the local search.


* **hsa00010_processed.graphml_bruteout2.pickle**

    Representation of the model returned by the local search component of scBONITA-RD


* **hsa00010_processed.graphml_equivs1.pickle**

    Equivalent rule set returned by the local search component of scBONITA-RD
    

### Load required pacakges

In [1]:
import pickle, os, os.path
from pathlib import Path
import networkx as nx
import glob
import pandas as pd

### Unpack pickle objects

#### *_out2.pickle: Representation of the model returned by the genetic algorithm component of scBONITA-RD

#### *_equivs1.pickle : Equivalent rule set returned by the local search component of scBONITA-RD

In [2]:
currentDir = os.path.dirname(os.path.abspath("Rule_Inference_With_ScBONITA.ipynb"))
equivsName = os.path.join(currentDir, 'data','hsa00010_processed.graphml_equivs1.pickle')
print(equivsName)
ERS=pickle.load(open(equivsName, "rb"))
print(ERS)

/gpfs/fs2/scratch/mpalshik/scBONITA-main/scBonita_package/src/scBONITA/data/hsa00010_processed.graphml_equivs1.pickle
[[[1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0], [0, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0, 0], [1, 1, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 1, 0, 0], [0, 1, 1, 0, 1, 0, 0], [1, 1, 1, 0, 1, 0, 0], [0, 0, 0, 1, 1, 0, 0], [1, 0, 0, 1, 1, 0, 0], [0, 1, 0, 1, 1, 0, 0], [1, 1, 0, 1, 1, 0, 0], [0, 0, 1, 1, 1, 0, 0], [1, 0, 1, 1, 1, 0, 0], [0, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 1, 0], [1, 1, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1, 0], [1, 0, 1, 0, 0, 1, 0], [0, 1, 1, 0, 0, 1, 0], [1, 1,

In [3]:
def ProcessLocalSearchOutput(allscData, graph, graphName):
    """
    Processes the output of the local search component of scBONITA and returns a text file containing plain-text Boolean rules in the ERS, and other information such as in-degree and out-degree for each node in the network.
    
    Parameters:
        graphName: name of the graphml file originally provided for rule inference
    
    Returns:
        A Pandas DataFrame with the following columns:
        'Node': name of the node in the network
        'In_degree': in-degree in the processed network
        'Out_degree': out-degree in the processed network
        'Equivs_len'
        'Graph': the provided graphName
        'Number_of_Nodes': number of nodes in 'graph'
        'importanceScore': the importance score of each node in 'graph' as calculated by scBONITA
        'AvgLocalError': the average error of the local search in scBONITA-RD. 
        'plainEquivs': the rules/equivalent rule set from the local search, in human-readable format
        'bitstringEquivs': the rules/equivalent rule set from the local search, in bitstring format - this is the internal representation of rules in scBONITA and may be ignored by users
        'minimalRules': the minimal rule set (minimizing the number of )
    
    """
    
    equivs=graphName+"_processed.graphml_equivs1.pickle"
    print(Path(graphName), Path(equivs))
    if Path(graphName).is_file(
    ) and Path(equivs).is_file() and Path(
            graphName + "_processed.graphml_importanceScores.csv").is_file():


        errorsName = glob.glob(str(graphName) + "*_localErrors1.pickle")[0]
        localErrors = pickle.load(open(errorsName, "rb"))

        allscData._singleCell__inherit(graph,
                          removeSelfEdges=False,
                          restrictIncomingEdges=False)
        equivs = pickle.load(
            open(graphName + "_processed.graphml_equivs1.pickle", "rb"))

        equivs = [tuple(equivs[i]) for i in range(0, len(equivs))]
        #print(equivs)
        equivs_len = [len(equivs[i]) for i in range(0, len(equivs))]

        nodeList = list(allscData.nodeList)
        minimalRules = allscData.processERS_minimalRule(
            graphName + "_processed.graphml_equivs1.pickle")
        plainEquivs = {}
        bitstringEquivs = {}
        minimalRulesNodes = {}
        for node in range(0, len(nodeList)):
            plainRules = []
            ers = equivs[node]
            for rule in ers:
                plainRules.append(
                    allscData._ruleMaker__writeNode(
                        allscData.nodeList.index(allscData.nodeList[node]),
                        rule, allscData))
            plainEquivs[nodeList[node]] = set(plainRules)
            bitstringEquivs[nodeList[node]] = set(
                tuple(ers[i]) for i in range(0, len(ers)))
            minimalRulesNodes[nodeList[node]] = allscData._ruleMaker__writeNode(
                node, minimalRules[allscData.individualParse[node]:allscData.
                                   individualParse[node + 1]], allscData)
        in_degree = [
            graph.in_degree(node)
            for node in nodeList
        ]
        out_degree = [
            graph.out_degree(node)
            for node in nodeList
        ]
        inOutRatio = [
            float(inDeg + 1) / (outDeg + 1)
            for inDeg, outDeg in zip(in_degree, out_degree)
        ]
        #Get importance score
        importanceScoreDF = pd.read_csv(
            graphName + "_processed.graphml_importanceScores.csv")
        importanceScoreDict = {}
        for n in list(importanceScoreDF['Node']):
            temp = importanceScoreDF[importanceScoreDF.Node.isin({n})]
            importanceScoreDict[n] = list(temp['Importance Score'])[0]

        tempdf = pd.DataFrame(list(
            zip(nodeList, in_degree, out_degree, inOutRatio, equivs_len)),
                              columns=[
                                  'Node', 'In_degree', 'Out_degree',
                                  'Equivs_len'
                              ])
        tempdf["Graph"] = graphName
        tempdf["Number_of_Nodes"] = str(len(nodeList))
        tempdf["importanceScore"] = tempdf['Node'].map(importanceScoreDict)
        tempdf["AvgLocalError"] = localErrors
        tempdf["plainEquivs"] = tempdf['Node'].map(plainEquivs)
        tempdf["bitstringEquivs"] = tempdf['Node'].map(bitstringEquivs)
        tempdf["minimalRules"] = tempdf['Node'].map(minimalRulesNodes)
        return(tempdf)



NameError: name 'allscData' is not defined

### Print out plain-text rules

In [4]:
equivsName = os.path.join(currentDir, 'data','hsa00010_processed.graphml_equivs1.pickle')
allscData = os.path.join(currentDir, 'data','trainingData.csvscTest.pickle')
print(allscData)
allscData = pickle.load(open(allscData, "rb"))
allscData.pathwayGraphs

/gpfs/fs2/scratch/mpalshik/scBONITA-main/scBonita_package/src/scBONITA/data/trainingData.csvscTest.pickle


{'hsa00010': <networkx.classes.digraph.DiGraph at 0x2b8ebed75a60>}

In [5]:
os.chdir(os.path.join(currentDir, 'data'))
temp = ProcessLocalSearchOutput(allscData, graph = allscData.pathwayGraphs["hsa00010"], graphName = "hsa00010.graphml")
temp.columns

hsa00010.graphml hsa00010.graphml_processed.graphml_equivs1.pickle
Nodelist: ['DLAT', 'GALM', 'DLD', 'BPGM', 'MINPP1', 'AKR1A1', 'TPI1', 'GPI', 'ADPGK', 'ALDH9A1', 'AHR', 'PDHB', 'GAPDH', 'PGM2', 'MDH1', 'MDH2', 'POR', 'ALDOA', 'ALDOC', 'ALDH2', 'ALDH1B1', 'ALDH3A2', 'ALDH3B1', 'ADH5', 'PDHA1', 'LDHA', 'LDHB', 'LDHC', 'PKM', 'ENO1', 'ENO2', 'ENO3', 'PGAM1', 'PFKL', 'PFKM', 'PFKP', 'FBP1', 'PGM1', 'HK1', 'HK2', 'HK3', 'HKDC1', 'G6PC3', 'PGK1', 'PCK2', 'ACSS2', 'ACSS1']


/software/anaconda3/2020.07/lib/python3.8/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/software/anaconda3/2020.07/lib/python3.8/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/software/anaconda3/2020.07/lib/python3.8/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/software/anaconda3/2020.07/lib/python3.8/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/software/anaconda3/2020.07/lib/python3.8/site-packages/scipy/st


Individual parse: [0, 7, 14, 21, 28, 31, 38, 41, 48, 49, 52, 53, 53, 60, 63, 64, 65, 68, 69, 70, 73, 76, 79, 82, 89, 89, 92, 95, 98, 105, 112, 119, 126, 133, 136, 139, 142, 145, 148, 155, 162, 169, 176, 183, 190, 190, 190, 190]

Nodelist: ['DLAT', 'GALM', 'DLD', 'BPGM', 'MINPP1', 'AKR1A1', 'TPI1', 'GPI', 'ADPGK', 'ALDH9A1', 'AHR', 'PDHB', 'GAPDH', 'PGM2', 'MDH1', 'MDH2', 'POR', 'ALDOA', 'ALDOC', 'ALDH2', 'ALDH1B1', 'ALDH3A2', 'ALDH3B1', 'ADH5', 'PDHA1', 'LDHA', 'LDHB', 'LDHC', 'PKM', 'ENO1', 'ENO2', 'ENO3', 'PGAM1', 'PFKL', 'PFKM', 'PFKP', 'FBP1', 'PGM1', 'HK1', 'HK2', 'HK3', 'HKDC1', 'G6PC3', 'PGK1', 'PCK2', 'ACSS2', 'ACSS1']

Node positions: [30, 21, 33, 14, 42, 28, 4, 34, 32, 16, 40, 45, 46, 8, 3, 20, 18, 25, 26, 23, 24, 12, 19, 22, 44, 38, 41, 39, 29, 36, 6, 43, 17, 5, 27, 35, 11, 13, 10, 37, 0, 9, 7, 15, 1, 2, 31]

PossibilityList: [[11, 24, 45, 46], [38, 39, 40, 41], [0, 11, 24], [12, 29, 30, 31, 43], [3, 32], [9, 19, 20, 21, 22], [17, 18], [8, 13, 33, 34, 35, 36, 37, 38, 39, 40

ValueError: 4 columns passed, passed data had 5 columns

### Plot ERS sizes

### Make/visualize networks with minimal rules, random rules, pre-specified text file of rules